<a href="https://colab.research.google.com/github/Rogerio-mack/work/blob/main/IA_FSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right"> 

<br>
<br>

# Inteligência Artificial
---


# FSM

Uma máquina de estados finitas pode ser implementada com simples encadeamentos de `ifs` embora implementações mais eficientes e elegantes possam e devem ser feitas.

![imagem](https://github.com/Rogerio-mack/work/blob/main/figuras/fsm_ends_aba.png?raw=true)

O FSM acima verifica se uma cadeia de caracteres termina por 'aba' e pode ser implementado como abaixo.

In [ ]:
def match(s):
  state = 0

  for c in s:
    # print('before', c, state)
    if state == 0:
      if c in ['a']:
        state = 1
        continue
    if state == 1:
      if c in ['b']:
        state = 2
      else:
        state = 0
      continue
    if state == 2:
      if c in ['a']:
        state = 1
      continue  
    # print('after', c, state)    

  if state == 1:
    print('It matches')
  else:
    print('It doesn\'t match')

  return

print(match('abc'))
print(match('ab'))
print(match('aba'))
print(match('baba'))        
print(match('xyzababa'))  
print(match('xyzababab'))   
print(match('xyzabaabab'))              
print(match('xyzabaababax'))    

It doesn't match
None
It doesn't match
None
It matches
None
It matches
None
It matches
None
It doesn't match
None
It doesn't match
None
It doesn't match
None


Mas formas mais estruturadas podem ser implementadas como a seguir.

# FSM: Expressão Regular - ab*c

Implementa o FSM abaixo que verifica se uma *string* corresponde à expressão regular `ab*c`.

![imagem](https://github.com/Rogerio-mack/work/blob/main/figuras/fsm_abc.png?raw=true)

Os estados são modelados como co-rotinas Python que executam um loop infinito no qual aceitam a entrada, decidem a transição e atualizam o estado atual do FSM. A função de transição pode ser tão simples como um conjunto de `ifs` como temos aqui ou um sistema bem mais complexo que poderia envolver uma função complexa de decisão.

A implementação abaixo empregar generators e coroutines de Python e você pode encontrar mais sobre esses recursos logo abaixo.

In [6]:
def start(fn):
    def wrapper(*args, **kwargs):
        v = fn(*args, **kwargs)
        v.send(None)
        return v
    return wrapper

In [7]:
class RegexFSM:
    def __init__(self):
        self.start = self._create_start()
        self.q1 = self._create_q1()
        self.q2 = self._create_q2()
        self.q3 = self._create_q3()
        
        self.current_state = self.start
        self.stopped = False
        
    def send(self, char):
        try:
            self.current_state.send(char)
        except StopIteration:
            self.stopped = True
        
    def does_match(self):
        if self.stopped:
            return False
        return self.current_state == self.q3

    @start
    def _create_start(self):
        while True:
            char = yield
            if char == 'a':
                self.current_state = self.q1
            else:
                break
    
    @start
    def _create_q1(self):
        while True:
            char = yield
            if char == 'b':
                self.current_state = self.q2
            elif char == 'c':
                self.current_state = self.q3
            else:
                break

    @start
    def _create_q2(self):
        while True:
            char = yield
            if char == 'b':
                self.current_state = self.q2
            elif char == 'c':
                self.current_state = self.q3
            else:
                break

    @start
    def _create_q3(self):
        while True:
            char = yield
            break

In [8]:
def grep_regex(text):
    evaluator = RegexFSM()
    for ch in text:
        evaluator.send(ch)
    return evaluator.does_match()

In [9]:
grep_regex("a")

False

In [10]:
grep_regex("ab")

False

In [11]:
grep_regex("ac")

True

In [12]:
grep_regex("abc")

True

In [13]:
grep_regex("aba")

False

In [14]:
grep_regex("abbbbbbbc")

True

In [15]:
grep_regex("abcc")

False

In [16]:
grep_regex("abcd")

False

In [17]:
grep_regex("bcbc")

False

# Generators

`Generators` em Python são funções retomáveis que geram valores enquanto alguém, ao chamar a `next` função, continua solicitando. Elas empregam o comando `yield` para retornar um valor a cada `next`.

## Exemplo 1

In [1]:
def simpleGeneratorFun():
    yield 1
    yield 2
    yield 3
  
for value in simpleGeneratorFun(): 
    print(value)

1
2
3


In [2]:
x = simpleGeneratorFun()
  
print(x.__next__())  
print(x.__next__())
print(x.__next__())

1
2
3


## Exemplo 2

In [3]:
# Generator
def print_even(test_list) :
	for i in test_list:
		if i % 2 == 0:
			yield i

# initializing list
test_list = [1, 4, 5, 6, 7]

# printing initial list
print ("The original list is : " + str(test_list))

# printing even numbers
print ("The even numbers in list are : ", end = " ")
for j in print_even(test_list):
	print (j, end = " ")


The original list is : [1, 4, 5, 6, 7]
The even numbers in list are :  4 6 

# Coroutines

`Coroutines` em Python, assim como os geradores, são funções retomáveis mas, em vez de gerar valores, elas os consomem valores em tempo real.  Elas também empregam o comando `yield` para consumir um valor.

As co-rotinas são generalizações de sub-rotinas. Eles são usados ​​para multitarefa cooperativa, em que um processo produz voluntariamente (doa) controle periodicamente ou quando ocioso, para permitir que vários aplicativos sejam executados simultaneamente. Ao contrário das sub-rotinas, as co-rotinas têm muitos pontos de entrada para suspender e retomar a execução. A co-rotina pode suspender sua execução e transferir o controle para outra co-rotina e pode retomar novamente a execução do ponto em que parou. Também não existe uma função principal para chamar co-rotinas em uma ordem particular e coordenar os resultados. Desse modo a co-rotinas parecem ideais para a implementação de FSM.

Antes de empregar uma co-rotina ela precisar ser *started*, o que pode ser feito com um `next` ou um `send(None)`. 

## Exemplo

In [4]:
def print_name(contains):
  print('Searching contains:{}'.format(contains))
  while True:
    name = (yield)
    if contains in name:
      print(name)
    else:
      print('None')


In [5]:
# calling coroutine, nothing will happen
corou = print_name("Dear")

# This will start execution of coroutine and
# Prints first line "Searching prefix..."
# and advance execution to the first yield expression

# corou.__next__() # or
corou.send(None)

# sending inputs
corou.send("Friend")
corou.send("Dear Friend")
corou.send("Friend, my Dear")


Searching contains:Dear
None
Dear Friend
Friend, my Dear
